# blog app 작성 2
## 1. 템플릿 상속하기
### a. template 상속이란?
* 동일한 정보/레이아웃을 사용하고자 할 때, 모든 파일마다 같은 내용을 반복해서 입력하지 않아도 된다.
* 수정할 부분이 생겨도 한번만 수정하면 된다.

### b. 기본 template html 생성하기
* 기본 템플릿은 웹사이트 내 모든 페이지에 확장되어 사용되는 기본 template이다.
* blog/template/blog/base.html 파일을 생성한다.
* post_list.html의 모든 내용을 복사해 넣는다.


### c. 기본 template (base.html)
* {% block content %} {% endblock %} 이 부분에 바뀌는 부분을 넣어준다.
```html
<body>
     <div class="page-header">
         <h1><a href="/">Django Girls Blog</a></h1>
     </div>
     <div class="content container">
         <div class="row">
             <div class="col-md-8">
                 {% block content %}
                 {% endblock %}
             </div>
         </div>
     </div>
</body>
```

### d. post_list.html 수정하기
* extends : 상속 받는다는 의미
```html
{% extends 'blog/base.html' %}
    {% block content %}
         {% for post in posts %}
             <div class="post">
                 <div class="date">
                     {{ post.published_date }}
                 </div>
                 <h1><a href="">{{ post.title }}</a></h1>
                 <p>{{ post.text|linebreaksbr }}</p>
             </div>
         {% endfor %}
    {% endblock %}
```

---
## 2. Post Detail (글 상세) 페이지 작성하기
### a. urls.py에 url 추가
* blog/urls.py

```python
from django.urls import path
from . import views
urlpatterns = [
    path('', views.post_list, name='post_list'),
    path('post/<int:pk>/', views.post_detail, name='post_detail'), 
    # name은 post_list.html의 href에 사용한다
]

```

### b. post_list.html에 post_detail 페이지 링크 추가
* post.pk는 Post 모델의 primary key이다.

```html
{% extends 'blog/base.html' %}
{% block content %}
     {% for post in posts %}
     <div class="post">
         <div class="date">
             {{ post.published_date }}
         </div>
         <h1><a href="{% url 'post_detail' pk=post.pk %}">{{ post.title }}</a></h1>
         <p>{{ post.text|linebreaksbr }}</p>
     </div>
     {% endfor %}
{% endblock %}
```

### c. post_detail() 함수를 views.py에 추가
* pk가 잘못되었으면 404에러를 호출
* 있으면 가져오기

```python
from django.shortcuts import render, get_object_or_404
from .models import Post
def post_detail(request, pk):
     post = get_object_or_404(Post, pk=pk)
     return render(request, 'blog/post_detail.html', {'post': post})

```

### d. post_detail.html 페이지 작성하기
* blog/template/blog/post_detail.html

```html
{% extends 'blog/base.html' %}
{% block content %}
     <div class="post">
         {% if post.published_date %}
             <div class="date">
                 {{ post.published_date }}
             </div>
         {% endif %}
         <h1>{{ post.title }}</h1>
         <p>{{ post.text|linebreaksbr }}</p>
     </div>
{% endblock %}

```

---
## 3. Django Form (글 추가)
### a. Form?
* model 클래스와 유사하게 Form 클래스를 정의
* 커스텀 Form 클래스
    * 입력 폼 HTML 생성 : .as_table(), .as_p(), .as_ul() 기본제공

### b. Form 처리
* 폼 처리 시에 같은 URL에서 GET/POST 로 나우어 처리
* GET방식으로 요청 : 입력 폼을 보여줍니다.
* POST방식으로 요청 : 데이터를 입력 받아 유효성 검증 과정을 거칩니다.
    * 검증 성공 : 데이터를 저장하고 SUCCESS URL로 이동
    * 검증 실패 : 오류 메세지와 함께 입력 폼을 다시 보여준다.
    
### c. Form 처리과정

![flow_chart](img/flow.PNG)

### 3-1.글추가 페이지 추가
### a. : Form 클래스 정의
* forms.py

```python
from django import forms
from .models import Post

# Model Form을 상속받는
class PostModelForm(forms.ModelForm):
    # ModelForm : Model과 연관된 Form을 만듦
    class Meta: # rule
        model = Post
        fields = ('title', 'text')
    # validate검사를 Modelform에서 할 수 없음
    # 그래서 Model에서 해주어야한다.
```



### b. post_edit.html 페이지 추가
* blog/templates/blog/post_edit.html
```html
{% extends 'blog/base.html' %}
{% block content %}
    <h1>New post</h1>
     <form method="POST" class="post-form">
         {% csrf_token %} <!-- 토큰이 없으면 403 에러가 난다. -->
         {{ form.as_p }}
         <button type="submit" class="save btn btn-default">Save</button>
     </form>
{% endblock %}
```

### c. post_new() 함수 만들기

* blog/views.py

```python
from .forms import PostForm
def post_new(request):
    form = PostForm()
    return render(request, 'blog/post_edit.html', {'form': form}) 
```

* blog/templates/blog/post_edit.html
    * 위에서 넘긴 form을 html에서 사용
    
    
```html
{% extends 'blog/base.html' %}
{% block content %}
    <h1>New post</h1>
    <form method="POST" class="post-form">
        {% csrf_token %}
        {{ form.as_p }}
        <button type="submit" class="save btn btn-default">Save</button>
    </form>
{% endblock %}

```




### d. urls.py에 추가
```python
# localhost:8000/post/new
path('post/new/', views.post_new, name='post_new'),
```

### e. base.html 에 글 추가 링크 추가

```html
<div class="page-header">
     <a href="{% url 'post_new' %}" class="plus">
        <span class="glyphicon glyphicon-plus"></span></a>
     <h1><a href="/">Django's Blog</a></h1>
</div>

```

* 원하는 대로 css도 작성해준다

```css
.plus, .plus:hover, .plus:visited {
    text-decoration: none; /* link 표시 안나게 하기! */
    color: #ffffff;
    font-size: 26pt;
    margin-right: 20px;
    float: right; /*오른쪽 으로 옮기기*/
}
```

### f. 필드 유효성 검사 함수 추가
* models.py 수정

```python
from django import forms

def min_length_3_validator(value):
    if len(value) < 3:
        raise forms.ValidationError('3글자 이상 입력해주세요.')
```

* migration 해준다

### g-1. Form 저장하기
* 등록 Form의 두 가지 상황
    * 첫번째 : 처음 페이지에 접속 했을 때, 새 글을 쓸 수 있게 Form이 비어있다.
        * 이때 Http method는 GET
    * 두번째 : Form에 입력된 데이터를 view페이지로 가지고 올 때 이다.
        * 이때 Http method는 POST
* blog/views.py
```python
from .forms import PostForm
def post_new(request):
    if request.method == "POST":
         form = PostForm(request.POST)
    else:
        form = PostForm()
    return render(request, 'blog/post_edit.html', {'form': form})
```

### g-2. redirect
* 새 글을 작성하고 post_detail 페이지로 이동한다.
* post_detail은 이동하고 싶은 view의 name이고, pk=post.pk 를 이용해서 값을 넘긴다.
* post는 새로 생성한 블로그 글입니다.

```python
from django.shortcuts import redirect

return redirect('post_detail', pk=post.pk)
```

### g-3 post_new 완성!
```python
# post 등록
def post_new(request):
    if request.method == 'POST': # save버튼을 눌렀을때,
        form = PostForm(request.POST)
        if form.is_valid(): # 유효성 검사
            print(form.cleaned_data)
            post = Post.objects.create(
                author=User.objects.get(username = request.user),
                                       published_date=timezone.now(),
                                       title=form.cleaned_data['title'],
                                       text=form.cleaned_data['text']) 
            # save 필요없다
            return redirect('post_detail', pk=post.pk) 
        # 저장하자 마자 post_detail로 바로 분기

    else : # http method == 'GET'
        form = PostForm() # 등록 form을 보여준다
    return render(request, 'blog/post_edit.html', {'form': form} )
```

---
## 4. 글 수정
### a. view.py에 post_edit() 함수 추가
```python
# Post 수정
def post_edit(request, pk):
    post = get_object_or_404(Post, pk=pk)
    if request.method == 'POST': # save버튼을 누름
        # 수정을 처리하는 부분
        form = PostModelForm(request.POST, instance=post)
        if form.is_valid():
            post = form.save(commit=False)
            post.author = User.objects.get(username = request.user)
            post.published_date = timezone.now()
            post.save()
            return redirect('post_detail', pk=post.pk)
    else : # 연필 버튼을 누름
        # 수정하기 전에 데이터를 읽어오는 부분
        form = PostModelForm(instance=post) # 데이터를 instance로 주기
    return render(request, 'blog/post_edit.html', {'form': form})
```

### b. urls.py 추가
```python
# localhost:8000/post/5/edit
    path('post/<int:pk>/edit', views.post_edit, name='post_edit')
```

### c. post_detail.html 에 글 수정 링크 걸기
```html
{% extends 'blog/base.html' %}

{% block content %}
      <div class="post">
          {% if post.published_date %}
            <div class="date">
                {{post.published_date}}
            </div>
          {% endif %}}
<!--          수정하기-->
          <a class = "btn btn-default" href="{% url 'post_edit' pk=post.pk %}">
              <span class = "glyphicon glyphicon-pencil"></span>
          </a>
<!--          여기까지 -->
          <h1> {{post.title}}</h1>
          <p> {{post.text|linebreaksbr }}</p>
      </div>
{% endblock %}
```

---
## 5. 글 삭제
### a. blog/views.py에 post_remove() 함수 추가
```python
# post 삭제
def post_remove(request, pk):
    post = get_object_or_404(Post, pk=pk)
    post.delete() # 삭제하기
    return redirect('post_list') # 지우자 마자 글 list로 이동
```

### b. urls.py 추가
```python
# localhost:8000/post/5/remove
    path('post/<int:pk>/remove', views.post_remove, name='post_remove' )
```

### c. post_detail.html에 페이지 삭제 링크 만들기
* blog/templates/blog/post_detail.html

```html
{% extends 'blog/base.html' %}

{% block content %}
      <div class="post">
          {% if post.published_date %}
            <div class="date">
                {{post.published_date}}
            </div>
          {% endif %}
          <a class = "btn btn-default" href="{% url 'post_edit' pk=post.pk %}">
              <span class = "edit-btn glyphicon glyphicon-pencil"></span>
          </a>
<!--          글 삭제 링크 추가하기-->
          <a class = "btn btn-default" href="{% url 'post_remove' pk=post.pk %}">
              <span class="glyphicon glyphicon-remove"></span>
          </a>
<!--          여기까지-->
          <h1> {{post.title}}</h1>
          <p> {{post.text|linebreaksbr }}</p>
      </div>
{% endblock %}
```

---
## 6. 로그인/로그아웃 처리하기
### a. @login_required 데코레이터
* 로그인 한 사용자만 접근하도록 post_new, post_edit등을 보호하는 것

```python
@login_required
def post_new(request):
    [...]
```

### b. 